For installation of **DSSEX** please follow [installation instructions](./readme.ipynb#Install-DSSEX).

Alternatively you can run ```%pip install dssex``` in a cell.

In [ ]:
import dssex
from egrid import create_objects, make_data_frames, check_frames, model_from_frames
import pandas as pd
pd.set_option('display.max_colwidth', None)

def print_frames(frames):
    """Prints pandas.DataFrame instances stored in a dict"""
    messages = frames.get('Message')
    if messages is not None:
        is_error = 1 < messages.level
        errors = messages.loc[is_error].message
        if not errors.empty:
            print('Errors\n')
            for _, error in errors.items():
                print()
                print(error)
            print()
    for key,df in frames.items():
        print()
        print(key)
        display(df)
        
def print_results(res):
    """Prints values returned from function dssex.estimate"""
    messages, vals = res
    if not messages.empty:
        print('\n\n')
        print('Messages')
        display(messages)
    last_index = len(vals) - 2
    for idx, df in enumerate(vals, -1):
        print('\n\n')
        print(f'{"RESULT >> " if idx==last_index else ""}Optimization Step: {idx}')
        for name in ['nodes', 'branches', 'injections']:
            print()
            print(name.title())
            display(df[name])    

# Power Flow Calculation
## Model
### Create

Add equipment and connectivity nodes:
- IDs of nodes start with **n**, IDs of slack nodes with **slack**
- IDs of nodes are left and right of branches and only at one side of injections

[Elements of Model](./help_elements.ipynb)

In [ ]:
schema00 = """
slack line_slack_00  n00
      y_lo=1k-1kj
      y_tr=1µ+1µj

n00 line_00_01 n01                              
    y_lo=2k-2kj
    y_tr=2µ+2µj
     
load00_0 n00
P10=12 Q10=7
Exp_v_p=1 Exp_v_q=1

cap00_1 n00
Q10=-3
Exp_v_q=2

load01_0 n01
P10=42
Q10=27
"""
frames00 = make_data_frames(create_objects(schema00))
print_frames(frames00)

### Check

In [ ]:
pd.DataFrame.from_records(check_frames(frames00), columns=['message','level'])

### Calculate Power Flow

In [ ]:
model00 = model_from_frames(frames00)
results00 = dssex.estimate(model00)

In [ ]:
print_results(results00)

## Model2
Data augmented with pseudo graphic (same model as above).
Characters **)(-<>|** are ignored. 

In [ ]:
schema01 = """
slack)-------line_slack_00--(n00)--------line_00_01-------(n01)
              y_lo=1k-1kj     |           y_lo=2k-2kj       |
              y_tr=1µ+1µj     |           y_tr=2µ+2µj       |
                              |                             |
# this line is a comment:     |  at least one blank line    |  no blank line between
# cap00_1 has a trailing      |  is required between lines  |  lines defining elements
# underscore in order to      |  defining branches,         |  and lines adding
# avoid a connection to n01   |  injections and nodes       |  attributes
                              |                             |
          _load00_0 <-------(n00)--|| cap00_1_            (n01)--> load01_0_
              P10=12 Q10=7             Q10=-3                       P10=42
              Exp_v_p=1                Exp_v_q=2                    Q10=27
              Exp_v_q=1
"""
frames01 = make_data_frames(create_objects(schema01))
print_frames(frames01)

In [ ]:
model01 = model_from_frames(frames01)
results01 = dssex.estimate(model01)

In [ ]:
print_results(results01)

## Transformer Taps, Shunt Capacitor Steps

In [ ]:
schema03 = """
      Tlink=taps
slack)-----------transformer_slack_00--(n00)--------line_00_01-------(n01)
                  y_lo=10k-10kj          |           y_lo=2k-2kj       |
                  y_tr=1µ+8µj            |           y_tr=2µ+2µj       |
                                         |                             |
                     _load00_0 <-------(n00)--|| cap00_1_            (n01)--> load01_0_
                       P10=12 Q10=7               Q10=-3                       P10=42
                       Exp_v_p=1                  Exp_v_q=2                    Q10=27
                       Exp_v_q=1

# line '#.' switches from pseudo graphic mode to footer mode used for input of additional objects
#.

# create tap-factor 'taps', 'taps' is associated to terminal 'slack - transformer_slack_00' by Tlink in pseudo graphic part
# 'taps' models a tap-changer
Deft(id=taps min=-9 max=9 value=-5)

# create scaling factor 'kq_cap', 'kq_cap' models steps of cap00_1
Defk(id=kq_cap min=0 max=3 value=2)

# associate scaling factor 'kq_cap' to reactive power Q of cap00_1
Klink(id_of_injection=cap00_1 id_of_factor=kq_cap part=q)
"""
frames03 = make_data_frames(create_objects(schema03))
print_frames(frames03)

In [ ]:
pd.DataFrame.from_records(check_frames(frames03), columns=['message','level'])

In [ ]:
model03 = model_from_frames(frames03)
results03 = dssex.estimate(model03)

In [ ]:
print_results(results03)

## Calculate Power Flow with PV-Generator
Scale reactive power in order to meet magnitude of voltage-setpoint.

In [ ]:
schema04 = """
                                                                       V=.995
slack)-----------transformer_slack_00--(n00)--------line_00_01-------(n01)--> load01_0
                  y_lo=10k-10kj          |           y_lo=2k-2kj       |       P10=200 Q10=40
                  y_tr=1µ+8µj            |           y_tr=2µ+2µj       |
                                         |                             |
                     _load00_0 <-------(n00)--|| cap00_1_            (n01)--((~)) gen01_0_
                       P10=400 Q10=140             Q10=-40                         P10=-180
                       Exp_v_p=1                   Exp_v_q=2                       Q10=-170
                       Exp_v_q=1
#.
# create scaling factor 'kq_gen'
Defk(id=kq_gen min=-1 max=1 value=0)

# associate scaling factor 'kq_gen' to reactive power Q of gen01_0
Klink(id_of_injection=gen01_0 id_of_factor=kq_gen part=q)
"""
frames04 = make_data_frames(create_objects(schema04))
print_frames(frames04)

In [ ]:
pd.DataFrame.from_records(check_frames(frames04), columns=['message','level'])

[step_params](./help_params.ipynb)

In [ ]:
model04 = model_from_frames(frames04)
step_params04 = [dict(objectives='V')]
results04 = dssex.estimate(model04, step_params=step_params04)

In [ ]:
print_results(results04)

## Calculate Tap Position
Voltage is given by a set-point.

In [ ]:
schema05 = """
      Tlink=taps                         V=1
slack)-----------transformer_slack_00--(n00)--------line_00_01-------(n01)--> load01_0
                  y_lo=10k-10kj          |           y_lo=2k-2kj               P10=200 Q10=40
                  y_tr=1µ+8µj            |           y_tr=2µ+2µj
                                         | 
                     _load00_0 <-------(n00)--|| cap00_1_
                       P10=400 Q10=140             Q10=-40
                       Exp_v_p=1                   Exp_v_q=2
                       Exp_v_q=1
#.
Deft(id=taps type=var min=-9 max=9 value=0)
"""
frames05 = make_data_frames(create_objects(schema05))
print_frames(frames05)

In [ ]:
pd.DataFrame.from_records(check_frames(frames05), columns=['message','level'])

In [ ]:
model05 = model_from_frames(frames05)
step_params05 = [dict(objectives='V')]
results05 = dssex.estimate(model05, step_params05)

In [ ]:
print_results(results05)